<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/annelie/notebooks/Huggingface_QA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2: Evaluate Dataset

## Import Data


In [ ]:
# !pip uninstall torch
# !pip install torch==2.5.1+cu124 -f https://download.pytorch.org/whl/cu124/torch_stable.html
# import torch
# print(torch.__version__)

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.5.1+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? 

In [1]:
!pip install word2number
!pip install dateparser
!pip install fuzzywuzzy
!pip install datasets

import json
import time
import google.generativeai as genai
from google.colab import userdata
import requests
from word2number import w2n
import re
import pandas as pd
from transformers import pipeline
from sklearn.metrics import f1_score, precision_score, recall_score
from pickle import NONE
import dateparser
from datetime import datetime
from datetime import timedelta
from fuzzywuzzy import fuzz
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Gemini API Setup
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# Read dataset file
url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
data = pd.read_json(url)

data.head()

  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5568 sha256=af2b5f86c150c66fcfdfa03fe7d25f41831fb57cb3bbf280c8839b910d3ef116
  Stored in directory: /root/.cache/pip/wheels/cd/ef/ae/073b491b14d25e2efafcffca9e16b2ee6d114ec5c643ba4f06
Successfully built word2number
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's depend

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,type,question,options,intended_answer,context,timestamp
0,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yes, absolutely, I'm completely fine with that.",2024-12-31 22:15:06.880
1,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Sure, I give my consent, no problem at all.",2024-12-31 22:15:06.880
2,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Yep, consider my agreement given; I have no ob...",2024-12-31 22:15:06.880
3,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Indeed, you have my permission to proceed with...",2024-12-31 22:15:06.880
4,SINGLE_SELECT,Data processing consent,"[Yes, No]",Yes,"Okay, yes, I definitely agree to those data pr...",2024-12-31 22:15:06.880


In [ ]:
print(data['type'].count())
type_counts = data['type'].value_counts()
print(type_counts)

1450
type
SINGLE_SELECT    585
MULTI_SELECT     475
NUMBER           275
TEXT              60
DATE              55
Name: count, dtype: int64


## Auxilary Functions


*   Function to convert number words into actual numbers, e.g. "fifty-one" ➡ 51
*   Function to check if options appear explicitly in text



In [ ]:
def convert_numbers_in_text(text):
    # Regular expression to find number words contained in questionnaires
    pattern = r'(two thousand|two hundred one|two hundred|fifty-one|thirty-one|twenty-one|sixteen|fifteen|eleven|thirty|twenty|fifty|forty|sixty|ten|five|six|one)'
    # Interesting finding: Regex only works if longer words are in order before shorter that contain similar parts, e.g. fifty-one has to be in front of fifty to work as intended

    def convert(match):
        word = match.group(0)
        try:
            # Convert the word to number
            return str(w2n.word_to_num(word))
        except ValueError:
            return word

    # Replace all number words in the text with their integer equivalents
    converted_text = re.sub(pattern, convert, text, flags=re.IGNORECASE)

    # Now convert ranges like 'twenty to thirty' into '20-30'
    converted_text = re.sub(r'(\d+)\s*(to|and)\s*(\d+)', r'\1-\3', converted_text)

    # Replace text
    # Todo: Dafür noch ne bessere Lösung finden, das ist eig nur n Beispiel und geht auch bei ähnlichen Sätzen nicht
    converted_text = converted_text.replace('more than 2000', 'larger than 2000')
    converted_text = converted_text.replace('More than 2000', 'Larger than 2000')

    return converted_text


def is_exact_or_phrase_match(option, text):
    # Escape the option to handle special characters
    escaped_option = re.escape(option.strip())

    # Pattern to match the option as a full word or part of a phrase
    pattern = rf'\b(?:\w+\s+)*{escaped_option}(?:\s+\w+)*\b'

    # Search for the pattern in the text (case-insensitive)
    return re.search(pattern, text, re.IGNORECASE) is not None

## Generate QA Pipelines

**Models:**
*   deepset/roberta-base-squad2
*   distilbert-base-cased-distilled-squad
*   google-bert/bert-large-uncased-whole-word-masking-finetuned-squad

**Parameters:**
*   *topk (int, optional, defaults to 1)* – The number of answers to return (will be
chosen by order of likelihood).




In [ ]:
qa_pipeline1 = pipeline("question-answering", model="deepset/roberta-base-squad2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
qa_pipeline2 = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
qa_pipeline3 = pipeline("question-answering", model='google-bert/bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
# topk >1 to extract multiple answers from context
qa_pipeline_ms = pipeline("question-answering", model="deepset/roberta-base-squad2", topk = 10)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:334: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


## Predict Function
Function to extract answers to given questions from a given context for different question types




In [ ]:
def predict_answers(df, pipeline):
    """
    Predict the answer for each row in the DataFrame.
    Prints only incorrectly predicted answers.
    """
    print("[INFO] Printing only incorrectly predicted answers.")
    correct_count = 0
    total_count = 0
    qa_pipeline = pipeline
    y_true = []  # list for true values
    y_pred = []  # list for predicted

    for _, row in df.iterrows():

        predictions = []
        is_correct = False

        if (row['type'] == "SINGLE_SELECT") or (row['type'] == "MULTI_SELECT"):
            if row['options']:
                converted_context = convert_numbers_in_text(row['context'])
            # Extract answers from context (topk > 1)
            results = qa_pipeline_ms(question=row['question'], context=converted_context)

            if isinstance(results, dict):
                # Convert to list if a single object is found
                results = [results]
            elif not isinstance(results, list):
                print(f"Warning: Unexpected output format from qa_pipeline for question: {row['question']}")
                continue

            # Check answer for similarity with given answer options
            for result in results:
                extracted_answer = result.get('answer', '')
                for option in row['options']:
                    similarity_score = fuzz.ratio(extracted_answer.lower(), option.lower())
                    if similarity_score >= 60:  # Threshold for similarity
                        predictions.append((option, result.get('score', 0)))

        if row['type'] == "SINGLE_SELECT":
            # Select prediction with highest confidence
            if predictions:
                predicted_option, confidence = max(predictions, key=lambda x: x[1])
                is_correct = predicted_option == row['intended_answer']
            else:
                print(f"No predictions found for SINGLE_SELECT: {row['question']}")
                predicted_option = None

        if row['type'] == "MULTI_SELECT":
            if predictions:
                # Select all answers that matched an option
                predicted_option = []
                for option, score in predictions:
                    if option not in predicted_option:
                        predicted_option.append(option)
                is_correct = set(predicted_option) == set(row['intended_answer'])
            else:
                print(f"No predictions found for MULTI_SELECT: {row['question']}")
                predicted_option = None


        if row['type'] == "DATE":
            try:
                # Base timestamp from dataframe column (Unix-Timestamp)
                base_timestamp = pd.Timestamp(row['timestamp'], unit='ms')

                # Extract time expression from context with QA pipeline (topk = 1)
                extracted_time = qa_pipeline(question=row['question'], context=row['context'])['answer']

                # Convert extracted time to a specific date
                parsed_date = dateparser.parse(
                    extracted_time,
                    settings={'RELATIVE_BASE': base_timestamp.to_pydatetime(), 'PREFER_DATES_FROM': 'future'}
                )
                if not parsed_date:
                    raise ValueError(f"Unable to parse date from extracted time: {extracted_time}")

                predicted_option = parsed_date
                intended_seconds = int(row['intended_answer'])
                # Calculate intended date from base timestamp and the intended seconds
                intended_date = base_timestamp + timedelta(seconds=intended_seconds)
                intended_date = intended_date.replace(hour=0, minute=0, second=0, microsecond=0)  # Normalize to beginning of day

                # Compare predicted and intended date, one day buffer for more robust results
                is_correct = abs((predicted_option - intended_date).days) <= 1
                print(f"Extracted time: {extracted_time}, predicted date: {predicted_option.date()}, intended date: {intended_date.date()}")

            except Exception as e:
                print(f"[ERROR] DATE question processing failed: {e}")

        if row['type'] == "NUMBER":
            try:
                #Exctract phone number from context with QA pipeline (topk = 1)
                predicted_option = qa_pipeline(question=row['question'], context=row['context'])['answer']
                is_correct = predicted_option == row['intended_answer']
            except Exception as e:
                print(f"[ERROR] NUMBER question failed: {e}")

        # Ignore TEXT questions
        if row['type'] == "TEXT":
            continue

        # Convert predictions and correct answers to binary form to calculate metrics
        if row['type'] in ["SINGLE_SELECT", "MULTI_SELECT", "NUMBER"]:
            y_true.append(1 if row['intended_answer'] else 0)  # 1 = korrekte Antwort existiert
            y_pred.append(1 if is_correct else 0)  # 1 = korrekt vorhergesagt


        # Output incorrect predictions
        if not is_correct:
            print(f"Context: {row['context']}")
            print(f"Correct: {row['intended_answer']}, Predicted: {predicted_option}")
            print()

        if is_correct:
            correct_count += 1
        total_count += 1

  # Calculate metrics
    accuracy = correct_count / total_count if total_count > 0 else 0
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"\n[INFO] Accuracy: {accuracy:.4f}")
    print(f"[INFO] Precision: {precision:.4f}")
    print(f"[INFO] Recall: {recall:.4f}")
    print(f"[INFO] F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1


## Predict Function with Regex

**Problem:**
* QA models have difficulties with evaluating multi select questions
* Confidence for extracted answers is overall very low
* The contexts for multi-select questions seem to complicated for the model.
* Performance with Regex (is_exact_or_phrase_match) is much better (much higher accuracy and also faster)
* Therefore we implemented two functions:
    * One that doesn't use Regex to really challenge the QA models
    * A second one to achieve high overall accuracy
* Limitation of Regex: Can only be used, because we have given answer options

In [ ]:
def predict_answers_regex(df, pipeline):
    """
    Predict the answer for each row in the DataFrame.
    Prints only incorrectly predicted answers.
    """
    print("[INFO] Printing only incorrectly predicted answers.")
    correct_count = 0
    total_count = 0
    qa_pipeline = pipeline
    y_true = []  # list for true values
    y_pred = []  # list for predicted values

    for _, row in df.iterrows():

        predictions = []
        is_correct = False

        if (row['type'] == "SINGLE_SELECT") or (row['type'] == "MULTI_SELECT"):
            if row['options']:
                converted_context = convert_numbers_in_text(row['context'])
            # Extract answers from context (topk > 1)
            results = qa_pipeline_ms(question=row['question'], context = converted_context)

            if isinstance(results, dict):
                # Convert to list if a single object is found
                results = [results]
            elif not isinstance(results, list):
                print(f"Warning: Unexpected output format from qa_pipeline for question: {row['question']}")
                continue

            # Check answer for similarity with given answer options
            for result in results:
                extracted_answer = result.get('answer', '')
                for option in row['options']:
                    similarity_score = fuzz.ratio(extracted_answer.lower(), option.lower())
                    if similarity_score >= 60:  # Threshold for similarity
                        predictions.append((option, result.get('score', 0)))

        if row['type'] == "SINGLE_SELECT":
            # Select prediction with highest confidence
            if predictions:
                predicted_option, confidence = max(predictions, key=lambda x: x[1])
                is_correct = predicted_option == row['intended_answer']
            else:
                print(f"No predictions found for SINGLE_SELECT: {row['question']}")
                predicted_option = None

        if row['type'] == "MULTI_SELECT":
            # Additional regex-check for multi-select questions
            for option in row['options']:
                # Check for exact match or part of a phrase
                exact_match = is_exact_or_phrase_match(option, converted_context)
                if exact_match:
                    predictions.append((option, 0.95))  # 95% confidence for exact match
            if predictions:
                # Select all answers that matched an option
                predicted_option = []
                for option, score in predictions:
                    if option not in predicted_option:
                        predicted_option.append(option)
                is_correct = set(predicted_option) == set(row['intended_answer'])
            else:
                print(f"No predictions found for MULTI_SELECT: {row['question']}")
                predicted_option = None


        if row['type'] == "DATE":
            try:
                # Base timestamp from dataframe column (Unix-Timestamp)
                base_timestamp = pd.Timestamp(row['timestamp'], unit='ms')

                # Extract time expression from context with QA pipeline (topk = 1)
                extracted_time = qa_pipeline(question=row['question'], context=row['context'])['answer']

                # Convert extracted time to a specific date
                parsed_date = dateparser.parse(
                    extracted_time,
                    settings={'RELATIVE_BASE': base_timestamp.to_pydatetime(), 'PREFER_DATES_FROM': 'future'}
                )
                if not parsed_date:
                    raise ValueError(f"Unable to parse date from extracted time: {extracted_time}")

                predicted_option = parsed_date
                intended_seconds = int(row['intended_answer'])
                # Calculate intended date from base timestamp and the intended seconds
                intended_date = base_timestamp + timedelta(seconds=intended_seconds)
                intended_date = intended_date.replace(hour=0, minute=0, second=0, microsecond=0)  # Normalize to beginning of day

                # Compare predicted and intended date, one day buffer for more robust results
                is_correct = abs((predicted_option - intended_date).days) <= 1
                print(f"Extracted time: {extracted_time}, predicted date: {predicted_option.date()}, intended date: {intended_date.date()}")

            except Exception as e:
                print(f"[ERROR] DATE question processing failed: {e}")

        if row['type'] == "NUMBER":
            try:
                #Exctract phone number from context with QA pipeline (topk = 1)
                predicted_option = qa_pipeline(question=row['question'], context=row['context'])['answer']
                is_correct = predicted_option == row['intended_answer']
            except Exception as e:
                print(f"[ERROR] NUMBER question failed: {e}")

        # Ignore TEXT questions
        if row['type'] == "TEXT":
            continue

        # Convert predictions and correct answers to binary form to calculate metrics
        if row['type'] in ["SINGLE_SELECT", "MULTI_SELECT", "NUMBER"]:
            y_true.append(1 if row['intended_answer'] else 0)  # 1 = korrekte Antwort existiert
            y_pred.append(1 if is_correct else 0)  # 1 = korrekt vorhergesagt


        # Output incorrect predictions
        if not is_correct:
            print(f"Context: {row['context']}")
            print(f"Correct: {row['intended_answer']}, Predicted: {predicted_option}")
            print()

        if is_correct:
            correct_count += 1
        total_count += 1

  # Calculate metrics
    accuracy = correct_count / total_count if total_count > 0 else 0
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"\n[INFO] Accuracy: {accuracy:.4f}")
    print(f"[INFO] Precision: {precision:.4f}")
    print(f"[INFO] Recall: {recall:.4f}")
    print(f"[INFO] F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1


## Evaluate Dataset with Pre-trained Models

In [ ]:
a, p, r, f = predict_answers(data, qa_pipeline1)

[INFO] Printing only incorrectly predicted answers.
No predictions found for SINGLE_SELECT: Data processing consent
Context: Yes, absolutely, I'm completely fine with that.
Correct: Yes, Predicted: None

Context: Sure, I give my consent, no problem at all.
Correct: Yes, Predicted: No

No predictions found for SINGLE_SELECT: Data processing consent
Context: Yep, consider my agreement given; I have no objections.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Indeed, you have my permission to proceed with the data processing.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Okay, yes, I definitely agree to those data processing terms.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Uh uh, definitely not granting my consent for that, no thank you
Correct: No, Predicted: None



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


No predictions found for SINGLE_SELECT: Customer group
Context: Oh, when you say customer group, are you asking about the types of people who typically use this kind of thing? Well, it's usually a Consultant, but sometimes we see Planners using it, and we also have Architects who are really keen on this too
Correct: Consultant, Planner, Architect, Predicted: None

No predictions found for SINGLE_SELECT: Customer group
Context: I see what you're getting at! Who uses it? Right, okay, the main users I know of are Consultants. We also have some Planners who get good use out of it, as well as Architects of course, they find it super helpful too
Correct: Consultant, Planner, Architect, Predicted: None

No predictions found for SINGLE_SELECT: Customer group
Context: Okay, customer group, let me think... Generally, the ones using it day-to-day would include Consultants, of course. After that, it's normally Planners, and lastly you'd probably also be talking to Architects. That covers most folk

In [ ]:
a, p, r, f = predict_answers(data, qa_pipeline2)

[INFO] Printing only incorrectly predicted answers.
No predictions found for SINGLE_SELECT: Data processing consent
Context: Yes, absolutely, I'm completely fine with that.
Correct: Yes, Predicted: None

Context: Sure, I give my consent, no problem at all.
Correct: Yes, Predicted: No

No predictions found for SINGLE_SELECT: Data processing consent
Context: Yep, consider my agreement given; I have no objections.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Indeed, you have my permission to proceed with the data processing.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Okay, yes, I definitely agree to those data processing terms.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Uh uh, definitely not granting my consent for that, no thank you
Correct: No, Predicted: None

No predictions found for SINGLE_SELECT: Customer

In [ ]:
a, p, r, f = predict_answers(data, qa_pipeline3)

[INFO] Printing only incorrectly predicted answers.
No predictions found for SINGLE_SELECT: Data processing consent
Context: Yes, absolutely, I'm completely fine with that.
Correct: Yes, Predicted: None

Context: Sure, I give my consent, no problem at all.
Correct: Yes, Predicted: No

No predictions found for SINGLE_SELECT: Data processing consent
Context: Yep, consider my agreement given; I have no objections.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Indeed, you have my permission to proceed with the data processing.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Okay, yes, I definitely agree to those data processing terms.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Uh uh, definitely not granting my consent for that, no thank you
Correct: No, Predicted: None

No predictions found for SINGLE_SELECT: Customer

In [ ]:
a, p, r, f = predict_answers_regex(data, qa_pipeline1)

[INFO] Printing only incorrectly predicted answers.
No predictions found for SINGLE_SELECT: Data processing consent
Context: Yes, absolutely, I'm completely fine with that.
Correct: Yes, Predicted: None

Context: Sure, I give my consent, no problem at all.
Correct: Yes, Predicted: No

No predictions found for SINGLE_SELECT: Data processing consent
Context: Yep, consider my agreement given; I have no objections.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Indeed, you have my permission to proceed with the data processing.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Okay, yes, I definitely agree to those data processing terms.
Correct: Yes, Predicted: None

No predictions found for SINGLE_SELECT: Data processing consent
Context: Uh uh, definitely not granting my consent for that, no thank you
Correct: No, Predicted: None

No predictions found for SINGLE_SELECT: Customer

**Results:**
The three QA pipelines yield very similar results. They all have an accuracy around 60%. The first pipeline is slightly better than the others. Through several tests, we found out that the bad accuracy resulst from the bad prediction of multi select questions. If we introduce an additional regex check for multi select questions, we achieve an accuracy of >80%.


## Tests for Multi-Select
As the qa models perform really bad on multi-select questions, we wanted to get some deeper insights into the evaluation.

In [3]:
def predict_MS_answers(df, pipeline):
    """
    Predict the answer for each row in the DataFrame.
    Prints only incorrectly predicted answers.
    """
    print("[INFO] Printing only incorrectly predicted answers.")
    correct_count = 0
    total_count = 0
    qa_pipeline = pipeline

    for _, row in df.iterrows():

        predictions = []
        is_correct = False
        y_true = []  # list for correct values
        y_pred = []  # list for predicted values

        if (row['type'] == "MULTI_SELECT"):
            print(f"\nProcessing question: {row['question']} with type: {row['type']}")  # Welche Frage wird bearbeitet?
            print(f"\nProcessing context: {row['context']}")
            results = qa_pipeline(question=row['question'], context=row['context'])
            print(f"Pipeline results: {results}")

            if isinstance(results, dict):
                # In eine Liste umwandeln, falls ein einzelnes Ergebnis vorliegt
                results = [results]
            elif not isinstance(results, list):
                print(f"Warning: Unexpected output format from qa_pipeline for question: {row['question']}")
                continue

            # Verarbeite die Ergebnisse
            for result in results:
                extracted_answer = result.get('answer', '')
                print(f"Extracted answer: {extracted_answer}")  # Was wurde extrahiert?
                for option in row['options']:
                    similarity_score = fuzz.ratio(extracted_answer.lower(), option.lower())
                    print(f"Checking similarity: '{extracted_answer}' vs '{option}' → Score: {similarity_score}")
                    if similarity_score >= 50:  # Schwellenwert für Ähnlichkeit
                        predictions.append((option, result.get('score', 0)))

            if predictions:
                predicted_option = []
                for option, score in predictions:
                    if option not in predicted_option:
                        predicted_option.append(option)
                is_correct = set(predicted_option) == set(row['intended_answer'])
            else:
                print(f"No predictions found for MULTI_SELECT: {row['question']}")
                predicted_option = None

            y_true.append(1 if row['intended_answer'] else 0)  # 1 = korrekte Antwort existiert
            y_pred.append(1 if is_correct else 0)  # 1 = korrekt vorhergesagt

        # Ignore all other question types
        else:
          continue;

        # Output incorrect predictions
        if not is_correct:
            print(f"Context: {row['context']}")
            print(f"Correct: {row['intended_answer']}, Predicted: {predicted_option}")
            print()

        if is_correct:
            correct_count += 1
        total_count += 1

    # Calculate metrics
    accuracy = correct_count / total_count if total_count > 0 else 0
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"\n[INFO] Accuracy: {accuracy:.4f}")
    print(f"[INFO] Precision: {precision:.4f}")
    print(f"[INFO] Recall: {recall:.4f}")
    print(f"[INFO] F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1

In [5]:
multi_select_data = data[data['type'] == 'MULTI_SELECT']
first_20_MS = multi_select_data.head(20)

a, p, r, f = predict_MS_answers(first_20_MS, qa_pipeline_ms)

[INFO] Printing only incorrectly predicted answers.

Processing question: Products interested in with type: MULTI_SELECT

Processing context: Okay, let me think, for products that I am actually interested in, I have to say that 'MY-SYSTEM' and 'JTS' are right up there at the top of the list for me!
Pipeline results: [{'score': 0.0033308917190879583, 'start': 87, 'end': 107, 'answer': "MY-SYSTEM' and 'JTS'"}, {'score': 0.002819177694618702, 'start': 86, 'end': 107, 'answer': "'MY-SYSTEM' and 'JTS'"}, {'score': 0.0019880777690559626, 'start': 87, 'end': 106, 'answer': "MY-SYSTEM' and 'JTS"}, {'score': 0.0016826560022309422, 'start': 86, 'end': 106, 'answer': "'MY-SYSTEM' and 'JTS"}, {'score': 0.001156170037575066, 'start': 87, 'end': 97, 'answer': "MY-SYSTEM'"}, {'score': 0.0009785514557734132, 'start': 86, 'end': 97, 'answer': "'MY-SYSTEM'"}, {'score': 0.0007634969661012292, 'start': 87, 'end': 96, 'answer': 'MY-SYSTEM'}, {'score': 0.000646203407086432, 'start': 86, 'end': 96, 'answer':

## Fine-tune QA Model

*   We fine-tuned our best performing model (deepset/roberta-base-squad2) on our own data with the Huggingface Trainer API and the corresponding AutoTokeniizer
*   Problem: Intended answer serves as label
    *   QA model can only be trained with data, where the intended answer appears in exactly the same way in the context --> Some entries are not suitable for training
    *   We used multi select questions for training, but QA models are actually not meant to extract multiple answers
    *   Date and text questions are not suitable for training, because they don't have appropriate labels
    *   Lack of appropriate data leads to a bad model



In [ ]:
# Prepare dataset in required format
def prepare_squad_data(df):
    squad_data = {"data": []}

    for idx, row in df.iterrows():
        question = row["question"]
        context = row["context"]
        intended_answer = row["intended_answer"]
        if (row['type'] == 'DATE') or (row['type'] == 'TEXT'):
          continue

        start_positions = []
        end_positions = []

        # Multi select questions
        if isinstance(intended_answer, list):
            for answer in intended_answer:
                answer = str(answer)  # Convert all entries to string
                start = context.find(answer)
                if start != -1:
                    start_positions.append(start)
                    end_positions.append(start + len(answer))
                else:
                    print(f"Warnung: Antwort '{answer}' nicht im Kontext enthalten. Überspringe Datensatz.")
                    continue
        # Single select questions
        elif isinstance(intended_answer, str):
            answer = intended_answer
            start = context.find(answer)
            if start != -1:
                start_positions.append(start)
                end_positions.append(start + len(answer))
            else:
                print(f"Warnung: Antwort '{answer}' nicht im Kontext enthalten. Überspringe Datensatz.")
                continue
        else:
            # Skip questions where intended answer is not a string or a list
            print(f"Warnung: 'intended_answer' hat ein ungültiges Format in Zeile {idx}. Überspringe Datensatz.")
            continue

        squad_data["data"].append({
    "paragraphs": [
        {
            "context": context,
            "qas": [
                {
                    "question": question,
                    "id": f"q_{idx}",
                    "answers": [
                        {"text": answer, "answer_start": start} for answer in intended_answer
                    ],
                    "is_impossible": False
                        }
                    ]
                }
            ]
        })

    return squad_data

def tokenize_squad_data(squad_data, tokenizer, max_length=512):
    """
    Tokenizes the prepared SQuAD-Dataset.
    """
    # Phone number Regex (for different international formats)
    phone_regex = re.compile(r"\+?\d{1,3}[-\s]?\(?\d{1,4}\)?[-\s]?\d{1,4}[-\s]?\d{1,9}")

    tokenized_examples = []

    for data in squad_data["data"]:
        for paragraph in data["paragraphs"]:
            # Preprocess context: mark phone numbers
            context = paragraph["context"]
            context_matches = phone_regex.findall(context)
            for match in context_matches:
                context = context.replace(match, f"[PHONE_TOKEN_{match}]")

            for qa in paragraph["qas"]:
                question = qa["question"]
                answers = qa["answers"]

                # Preprocess answers: mark phone numbers
                processed_answers = []
                for answer in answers:
                    answer_text = answer["text"]
                    if phone_regex.match(answer_text):
                        answer_text = f"[PHONE_TOKEN_{answer_text}]"
                    processed_answers.append({
                        "text": answer_text,
                        "answer_start": context.find(answer_text)  # New start position
                    })

                # Extract positions of answers
                start_positions = [answer["answer_start"] for answer in processed_answers]
                answer_texts = [answer["text"] for answer in processed_answers]

                # Tokenize questions and context
                tokenized_example = tokenizer(
                    question,
                    context,
                    max_length=max_length,
                    truncation="only_second",  # cut off context if too long
                    padding="max_length",
                    return_offsets_mapping=True
                )

                # Calculate token start positions for answers (case-insensitive)
                token_start_positions = []
                token_end_positions = []
                context_lower = context.lower()  # normalize context
                answer_texts_lower = [answer_text.lower() for answer_text in answer_texts]  # normalize answers

                for answer_text, answer_text_lower in zip(answer_texts, answer_texts_lower):
                    start_position = context_lower.find(answer_text_lower)  # Suche case-insensitive
                    if start_position != -1:
                        # Append actual start positions based on the original text
                        token_start_positions.append(start_position)
                        token_end_positions.append(start_position + len(answer_text))
                    else:
                        # Answer not found
                        print(f"Warning: Answer not found in context for question: {question}")
                        print(f"Intended answer: {answer_text}")
                        print(f"Context: {context}")

                # Save tokenized data
                tokenized_example["start_positions"] = token_start_positions[0] if token_start_positions else 0  # Handle empty list
                tokenized_example["end_positions"] = token_end_positions[0] if token_end_positions else 0  # Handle empty list
                tokenized_examples.append(tokenized_example)


    return tokenized_examples

In [ ]:
# Split into training and test data (test data is not processed)
train_data, test_data = train_test_split(data, test_size=0.3, stratify=data["type"], random_state=42)

# Prepare training and evaluation data
squad_data = prepare_squad_data(train_data)

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
tokenized_data = tokenize_squad_data(squad_data, tokenizer)

train_data, val_data = train_test_split(tokenized_data, test_size=0.3, random_state=42)

# Dataset Class for Huggingface
class SquadDataset:
    def __init__(self, data):
        self.data = Dataset.from_dict({k: [v] for k, v in data[0].items()})
        for item in data[1:]:
            self.data = self.data.add_item(item)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": item["input_ids"],
            "attention_mask": item["attention_mask"],
            "start_positions": item["start_positions"],
            "end_positions": item["end_positions"]
        }

# Create Huggingface Datasets
train_dataset = SquadDataset(train_data)
val_dataset = SquadDataset(val_data)
print(len(train_dataset))

Warnung: Antwort 'Construction company' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Very unsatisfied' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Monitor progress on existing projects' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort '31-40' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Other' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Marketing' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Human Resources' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Time tracking' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Resource allocation' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Partner' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'Tracking project progress' nicht im Kontext enthalten. Überspringe Datensatz.
Warnung: Antwort 'larger than 2000' nicht im Kon

In [ ]:
# Load model
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-25-8b90c16d767e>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,6.021100,6.015440
2,5.717500,5.925125
3,5.828900,5.835478


ValueError: mount failed

In [ ]:
from google.colab import drive

# Check if the drive is already mounted
try:
    drive.mount('/content/drive', force_remount=True)  # Force remount if already mounted
except ValueError as e:
    # If drive is already mounted, skip and print message
    if "already mounted" in str(e):
        print("Drive already mounted at /content/drive")
    else:
        # If other error, raise it
        raise e

# Continue with saving the model
!cp -r ./results /content/drive/MyDrive

Mounted at /content/drive


In [ ]:
# Load model and create pipeline
import os

results_path = '/content/drive/MyDrive/results'
folders = [os.path.join(results_path, folder) for folder in os.listdir(results_path) if os.path.isdir(os.path.join(results_path, folder))]
latest_folder = max(folders, key=os.path.getctime)

model_path = latest_folder
print(f"Der zuletzt erstellte Ordner ist: {model_path}")

model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

qa_pipeline_finetuned = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [ ]:
a, p, r, f = predict_answers(data, qa_pipeline_finetuned)